In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

    
torch.manual_seed(42)    
features = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Flatten(),
)

# Load dataset from folder data/raw
dataset = datasets.ImageFolder(root='data/raw')

ds_idx = list(range(len(dataset)))
train_idx, test_idx = train_test_split(ds_idx, test_size=0.15, shuffle=True, stratify=dataset.targets, random_state=42)

train_dataset = torch.utils.data.Subset(dataset, train_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)

In [31]:
np.array(dataset[0][0])

array([[[50, 59, 66],
        [53, 62, 69],
        [54, 63, 70],
        ...,
        [49, 59, 68],
        [49, 59, 68],
        [50, 60, 69]],

       [[50, 59, 66],
        [52, 61, 68],
        [54, 63, 70],
        ...,
        [51, 61, 70],
        [51, 61, 70],
        [51, 61, 70]],

       [[49, 58, 65],
        [52, 61, 68],
        [53, 62, 69],
        ...,
        [51, 61, 70],
        [50, 60, 69],
        [50, 60, 69]],

       ...,

       [[54, 65, 71],
        [56, 67, 73],
        [59, 70, 76],
        ...,
        [38, 45, 53],
        [40, 47, 55],
        [42, 49, 57]],

       [[57, 68, 74],
        [58, 69, 75],
        [60, 71, 77],
        ...,
        [39, 46, 54],
        [40, 47, 55],
        [41, 48, 56]],

       [[59, 70, 76],
        [60, 71, 77],
        [62, 73, 79],
        ...,
        [39, 46, 54],
        [39, 46, 54],
        [40, 47, 55]]], dtype=uint8)

In [112]:
X_train = features(torch.stack([x for x, _ in train_dataset]))
y_train = torch.tensor([y for _, y in train_dataset])
X_test = features(torch.stack([x for x, _ in test_dataset]))
y_test = torch.tensor([y for _, y in test_dataset])

X_train = [x.detach().numpy() for x in X_train]
X_test = [x.detach().numpy() for x in X_test]
y_train = y_train.detach().numpy()
y_test = y_test.detach().numpy()

In [113]:
svc = SVC()

params = {
    'C': np.logspace(-3, 3, 7),
    'gamma': np.logspace(-3, 3, 7),
}

rscv = RandomizedSearchCV(
    svc,
    params,
    cv=10,
    n_jobs=-1,
    verbose=3,
    random_state=42,
    scoring='accuracy',
)

In [114]:
rscv.fit(X_train, y_train)      

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END .............C=0.01, gamma=1000.0;, score=0.301 total time=  13.1s
[CV 2/10] END .............C=0.01, gamma=1000.0;, score=0.293 total time=  13.2s
[CV 3/10] END .............C=0.01, gamma=1000.0;, score=0.295 total time=  13.2s
[CV 5/10] END .............C=0.01, gamma=1000.0;, score=0.295 total time=  13.1s
[CV 4/10] END .............C=0.01, gamma=1000.0;, score=0.295 total time=  13.2s
[CV 6/10] END .............C=0.01, gamma=1000.0;, score=0.295 total time=  13.1s
[CV 8/10] END .............C=0.01, gamma=1000.0;, score=0.303 total time=  12.9s
[CV 7/10] END .............C=0.01, gamma=1000.0;, score=0.303 total time=  13.1s
[CV 1/10] END ..............C=1000.0, gamma=1.0;, score=0.932 total time=   3.4s
[CV 2/10] END ..............C=1000.0, gamma=1.0;, score=0.955 total time=   3.3s
[CV 3/10] END ..............C=1000.0, gamma=1.0;, score=0.970 total time=   3.4s
[CV 4/10] END ..............C=1000.0, gamma=1.

RandomizedSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
                   random_state=42, scoring='accuracy', verbose=3)

In [115]:
rscv.best_score_

0.960691501480975

In [ ]:
0.960691501480975

In [4]:
import onnx
torch.onnx.export(
    features,
    torch.randn(3, 3, 128, 128),
    'prep.onnx',
    verbose=False,
    input_names=['input'],
    output_names=['output'],
    opset_version=11
)
model = onnx.load('prep.onnx')
model.graph.input[0].type.tensor_type.shape.dim[0].dim_param = '?'
model.graph.output[0].type.tensor_type.shape.dim[0].dim_param = '?'
onnx.save(model, 'dynamic_model.onnx')

In [116]:
import onnxruntime as rt

session = rt.InferenceSession('dynamic_model.onnx')

In [ ]:
x = np.random.random((3, 128, 128, 3))
x = x.astype(np.float32)
res = session.run(['output'], {'input': x})
res

[array([[0.01531086, 0.        , 0.        , ..., 0.04222197, 0.03978221,
         0.04232738],
        [0.01507288, 0.        , 0.        , ..., 0.04179475, 0.04133177,
         0.04220761],
        [0.01482842, 0.        , 0.        , ..., 0.04063625, 0.04072065,
         0.04342537]], dtype=float32)]

In [ ]:
res[0].shape

(3, 4096)

In [123]:
import onnx_tool

m = onnx_tool.Model('dynamic_model.onnx')
m.graph.shape_infer({'input': np.zeros((1, 128, 128, 3))})

In [128]:
from skl2onnx import to_onnx

onnx_model = to_onnx(rscv.best_estimator_, X_train[0])

In [129]:
with open("svm.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())